<a href="https://colab.research.google.com/github/chirag-sharma-00/cs182-cv-project/blob/main/project_notebook_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm

In [ ]:
!rm -rf sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!tar -xf drive/MyDrive/tiny-imagenet-200.tar.xz

## Preliminary Analysis

In [ ]:
images = pd.read_csv("tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [ ]:
#takes a while to run
images_expanded = pd.DataFrame(columns = ['Id', 'label'])
for img in images['Id']: 
    classes = images[images['Id'] == img]['labels']
    
    for img_class in classes: 
        images_expanded = images_expanded.append({'Id': img, 'label' : img_class}, ignore_index = True)

In [ ]:
images_expanded.to_csv('words_expanded.csv')

In [ ]:
data_dir = pathlib.Path('tiny-imagenet-200/')
image_count = len(list(data_dir.glob('**/*.JPEG')))

In [57]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
CLASS_NAMES.sort()
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200

## Testing Inception v3 out of the box

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
import copy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = data_dir
# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "inception_v3"
# Number of classes in the dataset
num_classes = 200
# Batch size for training (change depending on how much memory you have)
batch_size = 8
# Number of epochs to train for
num_epochs = 3
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params
feature_extract = True

In [ ]:
model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
#change last layers in model to match tiny imagenet
model.AuxLogits.fc = nn.Linear(768, num_classes)
model.fc = nn.Linear(2048, num_classes)

In [ ]:
if torch.cuda.is_available():
  model.cuda()

In [ ]:
#inception expects input size 3*299*299
input_size = 299

In [ ]:
from PIL import Image

In [ ]:
#test on one image
input_image_one = Image.open("tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

#converts image to 3*299*299
input_tensor_one = preprocess(input_image_one)

#convert to batches
input_batch = torch.cat((input_tensor_one.unsqueeze(0), input_tensor_one.unsqueeze(0)))
input_batch = input_batch.to(device)

with torch.no_grad():
    output = model(input_batch)

In [ ]:
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([[0.6637, 0.4933, 0.4839, 0.4828, 0.5951, 0.4830, 0.4790, 0.3699, 0.4196,
         0.4716, 0.5252, 0.4348, 0.5166, 0.3835, 0.6465, 0.4121, 0.4759, 0.5003,
         0.5494, 0.5972, 0.5674, 0.4990, 0.6205, 0.5291, 0.4867, 0.5412, 0.5171,
         0.5869, 0.5255, 0.5191, 0.4546, 0.3775, 0.5631, 0.5286, 0.5682, 0.4727,
         0.4739, 0.4929, 0.5127, 0.5284, 0.5206, 0.4834, 0.4205, 0.4837, 0.5085,
         0.5380, 0.4609, 0.5679, 0.4848, 0.5580, 0.6147, 0.4397, 0.4967, 0.5189,
         0.5783, 0.5649, 0.5924, 0.4191, 0.4089, 0.3955, 0.5462, 0.3598, 0.5634,
         0.5617, 0.4330, 0.6110, 0.4566, 0.5135, 0.5771, 0.5243, 0.4364, 0.4826,
         0.5339, 0.4466, 0.5800, 0.4521, 0.5315, 0.4604, 0.4775, 0.5296, 0.4712,
         0.4664, 0.5415, 0.4248, 0.3972, 0.5106, 0.5120, 0.5031, 0.4978, 0.5566,
         0.3899, 0.4492, 0.5022, 0.4175, 0.5557, 0.4461, 0.5699, 0.4395, 0.5105,
         0.4616, 0.4858, 0.4421, 0.5587, 0.4735, 0.5379, 0.5235, 0.6288, 0.4668,
         0.5342, 0.5073, 0.6

In [ ]:
torch.topk(probabilities[0], 5)

torch.return_types.topk(values=tensor([0.6753, 0.6637, 0.6578, 0.6479, 0.6465], device='cuda:0'), indices=tensor([118,   0, 164, 195,  14], device='cuda:0'))

In [ ]:
labels = pd.read_csv("tiny-imagenet-200/wnids.txt", names = ['class_labels'], sep = '\t')
labels['predictions'] = probabilities[0].cpu()

In [ ]:
labels.sort_values(by = ['predictions'], ascending = False)

,class_labels,predictions
118,n04560804,0.675302
0,n02124075,0.663655
164,n01917289,0.657767
195,n12267677,0.647887
14,n02132136,0.646546
...,...,...
7,n09246464,0.369855
121,n07615774,0.368008
151,n01983481,0.361850
61,n03637318,0.359774


### Updating feature extraction to match our dataset

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=1, is_inception=True):
    if torch.cuda.is_available():
      model = model.cuda()
      
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training (no val for now)
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:

                # move to gpu
                inputs = inputs.to(device)
                labels.data = labels.data.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
def set_parameter_requires_grad(model, feature_extracting=True):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    if torch.cuda.is_available():
      model = model.cuda()

    accuracies = []
    model.eval()
    
    for phase in tqdm.tqdm(['train', 'val']):
        counter = 0
        running_corrects = 0
        running_total = 0

        for inputs, labels in tqdm.tqdm(dataloaders[phase]): 
            inputs = inputs.to(device)
            labels.data = labels.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            counter += 1
            
            running_corrects += torch.sum(preds == labels.data)
            running_total += len(preds)

            if counter == 100:
              break
        phase_acc = running_corrects / running_total
        print(phase_acc)
        accuracies.append(phase_acc)

    return accuracies
            

In [ ]:
##inception model to finetune
model_ft = models.inception_v3(pretrained=True)
set_parameter_requires_grad(model_ft, feature_extract)
# Handle the auxilary net, requires_grads automatically set to true 
num_ftrs = model_ft.AuxLogits.fc.in_features
model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
# Handle the primary net, requires_grad automatically set to true 
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
input_size = 299

In [ ]:
##out of the box model
scratch_model = models.inception_v3(pretrained=True)
scratch_model.fc = nn.Linear(scratch_model.fc.in_features, 200)
scratch_model.AuxLogits.fc = nn.Linear(scratch_model.AuxLogits.fc.in_features, num_classes)


#### Check accuracy of out of the box model on all labeled data

In [46]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Custom dataloader based on https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
class ValidationDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None):
    self.img_labels = pd.read_csv(annotations_file, sep='\t', names=['image', 'label', 'x1', 'y1', 'x2', 'y2'])
    self.img_dir = img_dir
    self.transform = transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = Image.open(img_path)
    image = image.convert('RGB')
    label = self.img_labels.iloc[idx, 1]
    label = np.where(label==CLASS_NAMES)[0][0]
    if self.transform:
        image = self.transform(image)
    return image, label

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'val_annotations.txt'), os.path.join(data_dir, 'val', 'images'), data_transforms['val'])
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}

In [47]:
out_box_acc = predict(dataloaders_dict, scratch_model)

  0%|          | 0/1250 [00:00<?, ?it/s]

tensor(0.0037, device='cuda:0')



100%|██████████| 2/2 [00:08<00:00,  4.01s/it]

tensor(0.0050, device='cuda:0')


In [48]:
out_box_acc

[tensor(0.0037, device='cuda:0'), tensor(0.0050, device='cuda:0')]

#### Retrain inception on transformed data and check accuracy on train/val data

In [52]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'val_annotations.txt'), os.path.join(data_dir, 'val', 'images'), data_transforms['val'])
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}

In [53]:
params_to_update = []
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [ ]:
##finetuned model accuracy
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)

Epoch 0/2
----------


In [ ]:

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
    'val': ValidationDataset(os.path.join(data_dir, 'val', 'val_annotations.txt'), os.path.join(data_dir, 'val', 'images'), data_transforms['val'])
}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2) for x in ['train', 'val']}

In [ ]:
params_to_update = []
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [ ]:
##finetuned model accuracy
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)

Epoch 0/2
----------
train Loss: 5.1868 Acc: 0.2296
val Loss: 3.1657 Acc: 0.3264

Epoch 1/2
----------
train Loss: 5.0573 Acc: 0.2375
val Loss: 3.1471 Acc: 0.3283

Epoch 2/2
----------
train Loss: 5.0072 Acc: 0.2400
val Loss: 3.1676 Acc: 0.3188

Training complete in 28m 9s
Best val Acc: 0.328300
